## Section B : Query Building

#### Loading Data in this Notebook

In [4]:
#!pip install sqlalchemy==1.3.9
#!pip install ipython-sql
%load_ext sql
%sql sqlite:///queue_system_sqlite3.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### 3. Counter Summary

As all calls have a queue_id so using calls instead of queues. Here, the `CASE` expression evaluates whether the condition is met for each row, and if so, it contributes 1 to the count; otherwise, it contributes 0.

In [75]:
%%sql
SELECT counter_id, 
    token_letter as serving_token, 
    COUNT(called_Date) as called, 
    SUM(CASE WHEN ended_at IS NULL THEN 1 ELSE 0 END) as serving, 
    SUM(CASE WHEN call_status_id = 1 THEN 1 ELSE 0 END) AS served, 
    SUM(CASE WHEN call_status_id = 2 THEN 1 ELSE 0 END) AS no_show
  FROM calls 
  WHERE called_date = '2024-02-14' 
  GROUP BY counter_id, serving_token

 * sqlite:///queue_system_sqlite3.db
Done.


counter_id,serving_token,called,serving,served,no_show
1,NR,2,0,1,1
1,W,35,0,26,9
2,NR,8,0,8,0
2,W,4,0,4,0
3,NR,4,0,4,0
3,T,13,0,11,2
3,W,15,0,9,6
5,NR,2,0,2,0
5,W,19,0,13,6
6,NR,27,0,20,7


##### For `serving`
As there was no data in the calls and queues tables which could directly be used to find whether a client was currently being served or not, because the entries were updated only after the calls ended, it was decided that only calls made on a particular day which had not ended yet were still being served. An example of this is given below.

In [79]:
%%sql
SELECT (CASE WHEN ended_at IS NULL THEN 1 ELSE 0 END) as serving, * FROM calls WHERE called_date = '2024-01-28' AND ended_at IS NULL

 * sqlite:///queue_system_sqlite3.db
Done.


serving,id,queue_id,service_id,counter_id,user_id,token_letter,token_number,called_date,started_at,ended_at,waiting_time,served_time,turn_around_time,created_at,updated_at,call_status_id
1,447,387,2,1,5,NR,84,2024-01-28,2024-01-28 17:39:44,None,02:08:04,None,None,2024-01-28 17:39:44,2024-01-28 17:39:44,None
1,451,392,2,9,11,NR,88,2024-01-28,2024-01-28 17:53:50,None,02:07:21,None,None,2024-01-28 17:53:50,2024-01-28 17:53:50,None
1,453,397,2,4,12,NR,90,2024-01-28,2024-01-28 18:00:47,None,01:58:58,None,None,2024-01-28 18:00:47,2024-01-28 18:00:47,None


In [80]:
%%sql
SELECT counter_id, 
    token_letter as serving_token, 
    COUNT(called_Date) as called, 
    SUM(CASE WHEN ended_at IS NULL THEN 1 ELSE 0 END) as serving, 
    SUM(CASE WHEN call_status_id = 1 THEN 1 ELSE 0 END) AS served, 
    SUM(CASE WHEN call_status_id = 2 THEN 1 ELSE 0 END) AS no_show
  FROM calls 
  WHERE called_date = '2024-01-28'
  GROUP BY counter_id, serving_token

 * sqlite:///queue_system_sqlite3.db
Done.


counter_id,serving_token,called,serving,served,no_show
1,NR,3,1,2,0
1,W,32,0,27,5
2,NR,8,0,8,0
2,W,28,0,21,7
3,NR,3,0,3,0
4,NR,8,1,6,1
4,T,18,0,13,5
4,W,11,0,8,3
6,NR,3,0,3,0
6,W,27,0,25,2


### 4. Service Summary

In [121]:
%%sql
SELECT queues.service_id, 
    queues.letter, 
    COUNT(queues.id) as visitor, 
    SUM(CASE WHEN queues.position <> 0 THEN 1 ELSE 0 END) as queued, 
    COUNT(calls.called_Date) as called, 
    SUM(CASE WHEN calls.ended_at IS NULL THEN 1 ELSE 0 END) as serving, 
    SUM(CASE WHEN calls.call_status_id = 1 THEN 1 ELSE 0 END) AS served, 
    SUM(CASE WHEN calls.call_status_id = 2 THEN 1 ELSE 0 END) AS no_show
  FROM calls, queues 
  WHERE calls.queue_id = queues.id AND calls.called_date = '2024-02-14' 
  GROUP BY queues.service_id, queues.letter

 * sqlite:///queue_system_sqlite3.db
Done.


service_id,letter,visitor,queued,called,serving,served,no_show
1,W,87,0,87,0,65,22
2,NR,79,0,79,0,68,11
3,T,13,0,13,0,11,2


#### Queued calls are not recorded in calls

In [ ]:
%%sql
SELECT * FROM queues WHERE queues.position <> 0

 * sqlite:///queue_system_sqlite3.db
Done.


id,service_id,number,called,reference_no,letter,name,email,phone,position,created_at,updated_at
471,1,24,0,nF5QK6YnB,W,None,None,None,1,2024-01-29 13:40:48,2024-01-29 17:57:43
472,1,25,0,k8I7uO9UX,W,None,None,None,2,2024-01-29 13:40:53,2024-01-29 17:57:43
473,1,26,0,muzF4vbvh,W,None,None,None,3,2024-01-29 13:41:04,2024-01-29 17:57:43
474,1,27,0,zLYitWtZ3,W,None,None,None,4,2024-01-29 13:41:10,2024-01-29 17:57:43
475,1,28,0,x1Vsub95v,W,None,None,None,5,2024-01-29 13:41:14,2024-01-29 17:57:43
476,1,29,0,GAmEotpAH,W,None,None,None,6,2024-01-29 13:41:19,2024-01-29 17:57:43
477,1,30,0,cQu6ZCCWN,W,None,None,None,7,2024-01-29 13:41:25,2024-01-29 17:57:43
479,1,32,0,kbM89VU3h,W,None,None,None,8,2024-01-29 13:41:39,2024-01-29 17:57:43
480,1,33,0,jZuBh6UEo,W,None,None,None,9,2024-01-29 13:41:48,2024-01-29 17:57:43
481,1,34,0,qTIzpMebd,W,None,None,None,10,2024-01-29 13:41:55,2024-01-29 17:57:43


In [ ]:
%%sql
SELECT calls.*, queues.position FROM calls, queues WHERE calls.queue_id = queues.id AND queues.position <> 0

 * sqlite:///queue_system_sqlite3.db
Done.


id,queue_id,service_id,counter_id,user_id,token_letter,token_number,called_date,started_at,ended_at,waiting_time,served_time,turn_around_time,created_at,updated_at,call_status_id,position


In [115]:
%%sql
SELECT COUNT(*) as total_calls FROM calls

 * sqlite:///queue_system_sqlite3.db
Done.


total_calls
3354


In [116]:
%%sql
SELECT COUNT(*) as total_queues FROM queues

 * sqlite:///queue_system_sqlite3.db
Done.


total_queues
3374


### 5. Service x Counter Summary

In [120]:
%%sql
SELECT queues.service_id, 
    queues.letter, 
    calls.counter_id,
    COUNT(queues.id) as visitor, 
    SUM(CASE WHEN queues.position <> 0 THEN 1 ELSE 0 END) as queued, 
    COUNT(calls.called_Date) as called, 
    SUM(CASE WHEN calls.ended_at IS NULL THEN 1 ELSE 0 END) as serving, 
    SUM(CASE WHEN calls.call_status_id = 1 THEN 1 ELSE 0 END) AS served, 
    SUM(CASE WHEN calls.call_status_id = 2 THEN 1 ELSE 0 END) AS no_show
  FROM calls, queues 
  WHERE calls.queue_id = queues.id AND calls.called_date = '2024-02-14' 
  GROUP BY queues.service_id, queues.letter, calls.counter_id

 * sqlite:///queue_system_sqlite3.db
Done.


service_id,letter,counter_id,visitor,queued,called,serving,served,no_show
1,W,1,35,0,35,0,26,9
1,W,2,4,0,4,0,4,0
1,W,3,15,0,15,0,9,6
1,W,5,19,0,19,0,13,6
1,W,6,10,0,10,0,10,0
1,W,8,4,0,4,0,3,1
2,NR,1,2,0,2,0,1,1
2,NR,2,8,0,8,0,8,0
2,NR,3,4,0,4,0,4,0
2,NR,5,2,0,2,0,2,0


### 6. Agent Summary

In [123]:
%%sql
SELECT users.name, 
    queues.letter, 
    COUNT(queues.id) as visitor, 
    SUM(CASE WHEN queues.position <> 0 THEN 1 ELSE 0 END) as queued, 
    COUNT(calls.called_Date) as called, 
    SUM(CASE WHEN calls.ended_at IS NULL THEN 1 ELSE 0 END) as serving, 
    SUM(CASE WHEN calls.call_status_id = 1 THEN 1 ELSE 0 END) AS served, 
    SUM(CASE WHEN calls.call_status_id = 2 THEN 1 ELSE 0 END) AS no_show
  FROM queues JOIN calls ON queues.id = calls.queue_id 
    JOIN users ON calls.user_id = users.id
  WHERE calls.called_date = '2024-02-14' 
  GROUP BY users.name, queues.letter

 * sqlite:///queue_system_sqlite3.db
Done.


name,letter,visitor,queued,called,serving,served,no_show
Ajaya Limbu,NR,7,0,7,0,7,0
Bishnu Pd Marasini,NR,27,0,27,0,20,7
Bishnu Pd Marasini,W,10,0,10,0,10,0
Chhiring Ghale,NR,4,0,4,0,4,0
Chhiring Ghale,T,13,0,13,0,11,2
Chhiring Ghale,W,15,0,15,0,9,6
Lokendra Sunar,NR,22,0,22,0,19,3
Lokendra Sunar,W,4,0,4,0,3,1
Mahima Kalikote,W,35,0,35,0,26,9
Rahul khadka,NR,9,0,9,0,8,1
